# Searching the Roman_DESC Sims

The web server at TODO_URL has indexes of the transient truth tables and the Roman images.  (Hopefully in the not-too-distant future it will have the Rubin images as well.)

Search it by hitting the URL followed by `/<command>`, where `<command>` is one of `findimages`, ROB TODO.  You can pass additional parameters by adding one or more "/<kw>=<val>" to the end of the URL (sparating each kw/val pair with a slash), or sending a dictionary in the POST data.  Examples of both of these are below.
    
## Finding Images
    
The URL for finding images is `/findimages`.  You can filter on the following keywords:
* `num` — the pointing number
* `pointing_ra` — the RA of the pointing itself (not of any individual SCA)
* `pointing_dec` — the Dec of the pointing itself
* `exptime` — the time of the exposure
* `mjd` — the MJD of the exposure
* `pa` — the position angle of the exposure
* `scanuma` — the number of the SCA within the exposure
* `ra` — the central RA of the SCA (this is not useful, because you would have to match the float value exactly)
* `dec` — the central Dec of the SCA (also not useful)
* `containing` — a (ra,dec) (in decimal degrees): find images that include this position on the sky (this _is_ useful)
    
Examples are below.
    
## Finding Transients
    
The URL for finding transients is `/findtransients`.  These search the parquet files with information about transients injected into the simulations.  Right now, you can't get "truth" lightcurves from the server, just information about the transients as a whole.
    
The fields you get back when searching a transient may include:
* `id` — int, SNANA id of the transient
* `healpix` — int, parsed from the parquet filename
* `ra` — float, decimal degrees
* `dec` — float, decimal degrees
* `host_id` — int, SNANA host ID (not sure how this corresponds to IDs in other tables)
* `gentype` — int, SNANA gentype
* `model_name` — str, name of the modle
* `start_mjd` — float, first mjd that the transient was injected into the sims.  (May have been well below detection!)
* `end_mjd` — float, last mjd that the transient was injected into the sims.  (May have been well below detection!)
* `z_cmb` — float
* `mw_ebv` — float, but I think this is always a bogus number as the MW reddening was not applied at the transient truth stage
* `mw_extinction_applied` — bool, I think always False
* `AV` — float, TBD
* `RV` — float, TBD
* `v_pec` — float, TBD
* `host_ra` — float, decimal degrees
* `host_dec` — float, decimal degrees
* `host_mag_*` — float, where `*` is `g`, `i`, or `f` (note: lowercase `f`, not uppercase as in the parquet files!)
* `host_sn_sep` — float, units ??
* `peak_mjd` — float, mjd of the nominal (B-band? probably model dependent definition) peak of the transient lightcurve
* `peak_mag_*` — float, where `*` is `g`, `i`, or `f`
* `lens_dmu` — float (always bogus?)
* `lens_dmu_applied` — bool (always False?)
* `model_params` — dict of parameters saved from the particular model for this transient
    
You can search on most of these parameters, though in many cases that would not be meaningful.  Apply search criterion for transients in one of two ways.  First, you can append `/<kw>=<val>` to the end of the URL (with as many slash-separated `kw/val` pairs as you want).  Second, you can include `{kw:val}` in the POST dictionary you send to the server.  Examples of both of these are below.  For numeric parameters, you can append `_min` or `_max` to the end of `kw` to do a range search.  (E.g., if you wanted all transients between `0.15≤z≤0.18`, you could do `/findtransients/z_cmb_min=0.15/z_cmb_max=0.18`.)

# Examples

The only python library you need is `requests`.  I also import `pandas` for convenience in showing the results of example searches.

Below, we set up the variable `server_url` to be used throughout the rest of the examples.  We also create a `requests` Session in `req` for use throughout the rest of the examples.

In many examples below, I check the `status_code` of the value returned by any requests to the webserver.  If this is not 200 (which is HTTP-speak for a normal return), it means something went wrong.  In that case, the `text` field of the return value may have useful information; if not to you, then potentially to give to Rob for debugging purposes.

If the `status_code` is 200, then all is well, and you can call the `json()` method on the return value to get a dictionary of values.  This dictionary is formatted so that it's suitable for immediate import into a pandas DataFrame (as you will see in the examples below).

In [1]:
import requests
import pandas

# Configure the server URL.  NOTE: right now this is a temporary development
#  server.  The actual URL once we're in production will be different.  So 
#  don't write this nasty url down anywhere.
server_url = 'https://webserver.romansnpit-roman-desc-simdex.development.svc.spin.nersc.org'
req = requests.Session()
req.verify = False        # Usually want this true, but if you're using a test server with a self-signed cert, need False

## Examples: Finding Images

### Image Example 1: finding all images containing a point

In the first way, we'll encode the ra and dec in the URL itself.  (If we're still using the dev server, we'll get a `InsecureRequestWarning`; don't worry about that, it's just because the dev server doesn't have a proper SSL certificate.  The production one will.)

In [2]:
result = req.post( f'{server_url}/findimages/containing=(7.3152,-45.4391)' )
if result.status_code != 200:
    raise RuntimeError( f"Got status code {result.status_code}\n{result.text}" )

# If all went well (status_code was 200 and there was no exception), then you can pull
#  out the returned data with result.json()
df = pandas.DataFrame( result.json() )
print( f"Got {len(df)} images." )
display( df )
# Somebody somewhere seems to have sorted the columns alphabetically, which isn't really
#  what you would want, but oh well, they're all there

/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webserver.romansnpit-roman-desc-simdex.development.svc.spin.nersc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Got 0 images.


,boredec,borera,dec,dec_00,dec_01,dec_10,dec_11,exptime,filter,mjd,pa,pointingnum,ra,ra_00,ra_01,ra_10,ra_11,scanum


If you prefer, instead of packing all of the search parameters in slash-separated keyword-value pairs in the URL, you can send POST data using the `json=` parameter:

In [3]:
result = req.post( f'{server_url}/findimages', json={ 'containing': [ 7.3152, -45.4391 ] } )
if result.status_code != 200:
    raise RuntimeError( f"Got status code {result.status_code}\n{result.text}" )
    
df = pandas.DataFrame( result.json() )
print( f"Got {len(df)} images" )
display( df )

Got 0 images


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webserver.romansnpit-roman-desc-simdex.development.svc.spin.nersc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,boredec,borera,dec,dec_00,dec_01,dec_10,dec_11,exptime,filter,mjd,pa,pointingnum,ra,ra_00,ra_01,ra_10,ra_11,scanum


## Examples: Finding Transients

In [11]:
# Find all transients between redshift 0.15 and 0.16
result = req.post( f'{server_url}/findtransients/z_cmb_min=0.15/z_cmb_max=0.16' )
if result.status_code != 200:
    raise RuntimeError( f"Got status code {result.status_code}\n{result.text}" )
    
df = pandas.DataFrame( result.json() )
print( f"Got {len(df)} transients" )
display( df )

Got 381 transients


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webserver.romansnpit-roman-desc-simdex.development.svc.spin.nersc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,av,dec,end_mjd,gentype,healpix,host_dec,host_id,host_mag_f,host_mag_g,host_mag_i,...,mw_extinction_applied,peak_mag_f,peak_mag_g,peak_mag_i,peak_mjd,ra,rv,start_mjd,v_pec,z_cmb
0,-9.0,-39.580721,63570.0,99,9923,-39.580380,9923000008570,19.982,21.460,20.694,...,False,19.011585,19.011585,19.011585,None,10.079961,-9.0,63550.0,0.0,0.1597
1,-9.0,-39.074532,63570.0,99,9923,-39.074525,9923000009970,22.605,23.775,23.192,...,False,18.583853,18.583853,18.583853,None,9.658906,-9.0,63550.0,0.0,0.1563
2,-9.0,-39.468878,62128.0,32,9923,-39.468887,9923000010017,20.079,21.592,20.813,...,False,24.572828,23.391932,23.388239,None,10.019425,-9.0,61896.0,0.0,0.1597
3,-9.0,-39.543361,61774.0,32,9923,-39.543640,9923000008073,19.549,20.943,20.211,...,False,26.009110,24.400398,24.820340,None,10.006852,-9.0,61622.0,0.0,0.1591
4,-9.0,-39.028209,63267.0,10,9923,-39.028204,9923000008350,19.056,21.078,20.139,...,False,22.196590,19.921576,20.196531,None,9.869027,-9.0,63056.0,0.0,0.1594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,-9.0,-43.793157,63026.0,32,10307,-43.793111,10307000004025,22.590,23.755,23.175,...,False,23.119951,21.672802,21.937930,None,8.900338,-9.0,62792.0,0.0,0.1552
377,-9.0,-42.400943,61866.0,10,10307,-42.400961,10307000003819,18.650,21.090,19.966,...,False,22.161100,20.368229,20.403765,None,10.668302,-9.0,61499.0,0.0,0.1575
378,-9.0,-42.673976,62701.0,10,10307,-42.673889,10307000002794,20.818,22.156,21.468,...,False,22.172995,20.448359,20.387297,None,11.074662,-9.0,62334.0,0.0,0.1566
379,-9.0,-44.764562,63268.0,21,10307,-44.764505,10307000002194,20.624,21.933,21.263,...,False,24.003310,23.862898,23.451912,None,9.354777,-9.0,63182.0,0.0,0.1516
